## Introduction

According to [the wikipedia article](https://en.wikipedia.org/wiki/Weapon_target_assignment_problem) the about the weapon target assignnment problem (WTA) can be formulated as follows:

Given a number of weapons and a number of targets. The weapons are of type $i=1,\ldots ,m$ and $W_i$ denotes the number available weapons of type $i$. Similarly, there are $j=1,\ldots ,n$ targets with value $V_j$. Any of the weapons can be assigned to any target. Each weapon type has a certain probability of destroying each target, given by $p_{ij}$.


**Remark:** as this is a non linear problem, we include a brief section about implementing non linear models in pyomo and some modeling tips for nlp.


## math formulation 

The WTA can be formulated as a non linear integer programm as follows:

$$
\begin{array}{llc}
\min & \sum_j (V_j \prod_i q_{ij}^{x_{ij}}) & \\
s.t. & \sum_j x_{ij} \leq W_i &, \forall i \\
     & x_{ij} \in \mathbb{N}
\end{array}
$$

### objective

minimizing the expected survival value or equivalently maximizing the expeced damange

### (decision) variable

- $x_{ij}$ is the number of weapons of type $i$ assigned to target $j$

### parameter

- $p_{ij}$ chance that one weapon $i$ destroys target $j$
- $q_{ij}:=(1-p_{ij})$ survival probability of target $j$ for weapon $i$

### constraints

Dont assign more weapons of type $i$ than available, i.e. $W_i$

### assumptions

we see that the model uses statistical independency of the target survival probability, when multiple weapons are assigned.

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from itertools import product

## example

We consider the following example (from [the wikipedia on the weapon target assignment problem](https://en.wikipedia.org/wiki/Weapon_target_assignment_problem))

In [ ]:
data = {
'problem_data' : {
    'weapon_num': {'ground':5,'air':2, 'sea':1},
    'target_val': {'T1':5,'T2':10,'T3':20},
    'success_probability': pd.DataFrame(data = {'T1':[0.3,0.1,0.4],
                                                'T2':[0.2,0.6,0.5], 
                                                'T3':[0.5,0.5,0.4]},
                                        index = ['ground','air', 'sea'])
},
    'model_name': 'weapon assignment problem',
    'solver': 'scip'
}
# for easier handling assign
daten = data['problem_data']

In [ ]:
print('Given weapons and their availibility:')
daten['weapon_num']

Given weapons and their availibility:


{'ground': 5, 'air': 2, 'sea': 1}

In [ ]:
print('Given targets and their value:')
daten['target_val']

Given targets and their value:


{'T1': 5, 'T2': 10, 'T3': 20}

In [ ]:
print('success probabilities:')
display(daten['success_probability'])

success probabilities:


,T1,T2,T3
ground,0.3,0.2,0.5
air,0.1,0.6,0.5
sea,0.4,0.5,0.4


## Implementation

As the used model for the weapon assignment problem is nonlinear lets recall:

### Pyomo expression for non linear models

Write down a non linear problem in pyomo is easy:

| operation | operator | example |
|----------|----------|----------|
| multiplication    | *   | expr = model.x * model.y   |
| division   | /   | expr = model.x / model.y  |
| exponentiation | **   | expr = (model.x+2.0)**model.y  |
| in-place multiplication | *=   | expr *= model.x   |
| in-place division | /=   | expr = /= model.y  |
| in-place exponentiation | **=   | expr **= model.x   |

moreover there more supported functions part of the pyomo package, e.g. trigonometric functions:

- https://static1.squarespace.com/static/5492d7f4e4b00040889988bd/t/57bd0faad482e927298cca8f/1472008110099/5_Nonlinear.pdf

### Some modeling tips

- recall many mathematical functions have a valid domain and evaluation outside of their domain caueses errors (-> bounds and innitilization)
- as solvers uses 1st and 2nd derivatives check bounds also w.r.t. derivatives ,e.g. for $f(x)=\sqrt{x}$ we have $f(0)$ is valid but $\frac{\partial}{\partial_x}f(0)$ is not
- scale model to avoid variables, contraints, derivatives with different scales


### implementation remark

#### solver supports minlp

Some Sovlers can handle MINLPs like SCIP, hence after defining an non linear model in pyom, we can solve it as it were a linear model
```
# define model
model = pyo.ConcreteModel()
...
# choose solver an apply it
solver = pyo.SolverFactor('scip')
solver.solve(model)
```

But Pyomo comes with `MindtPy` a mixed-integer nonlinear decomposition toolbox, which allows using decomposition algorithms to solve MINLP. (c.f. upcomming blog post)

#### using decomposition algorithms from `MindtPy`

```
solver = pyo.SolverFactory('mindtpy')
solver.solve(model, mip_solver='glpk', nlp_solver='ipopt') 
```

reference: https://pyomo.readthedocs.io/en/stable/contributed_packages/mindtpy.html

In [ ]:
def wta_scip(data):
    # assign data to more handleble structures
    daten = data['problem_data']
    probs = daten['success_probability']
    
    # create model instance
    m = pyo.ConcreteModel(data['model_name'])
    
    # sets
    m.I = pyo.Set(initialize = daten['weapon_num'].keys(), doc = 'weapon types')
    m.J = pyo.Set(initialize = daten['target_val'].keys(),doc = 'targets')
    # decision variable
    m.x = pyo.Var(m.I, m.J, domain = pyo.NonNegativeIntegers,
                  doc = 'number of weapons of type i assigned to target j')
    # parameter
    @m.Param(m.I, doc = 'upper limit for weapon type i')
    def W(m,i):
        return daten['weapon_num'][i]
    @m.Param(m.J, doc = 'target value')
    def V(m,j):
        return daten['target_val'][j]
    @m.Param(m.I, m.J, doc = 'sucess probability when assign weapon i to target j')
    def p(m,i,j):
        return probs.loc[probs.index == i, j].values[0]
    @m.Param(m.I, m.J, doc = 'survival probability (1-p_ij)')
    def q(m,i,j):
        return 1 - m.p[i,j]
    
    # constraints
    @m.Constraint(m.I, doc = 'available number of weapons of type i')
    def c1(m,i):
        return m.W[i] >= pyo.quicksum(m.x[i,j] for j in m.J) 
    
    # Objective
    ## objective expression
    m.objective = pyo.quicksum(m.V[j] * pyo.prod(m.q[i,j]**m.x[i,j] for i in m.I)  for j in m.J)
    ## add objective to model
    m.OBJ = pyo.Objective(expr = m.objective, sense = pyo.minimize)
    
    # define solver
    solver = pyo.SolverFactory('scip')
    solver.solve(m)
    
    return m

In [ ]:
def wta(data):
    # assign data to more handleble structures
    daten = data['problem_data']
    probs = daten['success_probability']
    
    # create model instance
    m = pyo.ConcreteModel(data['model_name'])
    
    # sets
    m.I = pyo.Set(initialize = daten['weapon_num'].keys(), doc = 'weapon types')
    m.J = pyo.Set(initialize = daten['target_val'].keys(),doc = 'targets')
    # decision variable
    m.x = pyo.Var(m.I, m.J, domain = pyo.NonNegativeIntegers,
                  doc = 'number of weapons of type i assigned to target j')
    # parameter
    @m.Param(m.I, doc = 'upper limit for weapon type i')
    def W(m,i):
        return daten['weapon_num'][i]
    @m.Param(m.J, doc = 'target value')
    def V(m,j):
        return daten['target_val'][j]
    @m.Param(m.I, m.J, doc = 'sucess probability when assign weapon i to target j')
    def p(m,i,j):
        return probs.loc[probs.index == i, j].values[0]
    @m.Param(m.I, m.J, doc = 'survival probability (1-p_ij)')
    def q(m,i,j):
        return 1 - m.p[i,j]
    
    # constraints
    @m.Constraint(m.I, doc = 'available number of weapons of type i')
    def c1(m,i):
        return m.W[i] >= pyo.quicksum(m.x[i,j] for j in m.J) 
    
    # Objective
    ## objective expression
    m.objective = pyo.quicksum(m.V[j] * pyo.prod(m.q[i,j]**m.x[i,j] for i in m.I)  for j in m.J)
    ## add objective to model
    m.OBJ = pyo.Objective(expr = m.objective, sense = pyo.minimize)
    
    # define solver
    pyo.SolverFactory('mindtpy').solve(m, mip_solver = 'cbc', nlp_solver = 'ipopt')
    
    return m

In [ ]:
#extract solution
def extract_solution(m):
    df = pd.DataFrame(index = data['problem_data']['weapon_num'].keys(),
                      columns = data['problem_data']['target_val'].keys())
    for i,j in product(m.I,m.J):
        df.at[i,j] = pyo.value(m.x[i,j])
        
    survival_value = pyo.value(m.OBJ)
    return {'assignment': df, 'survival_value': survival_value}

In [ ]:
%%time
m_scip = wta_scip(data)

CPU times: user 41 ms, sys: 14.8 ms, total: 55.9 ms
Wall time: 139 ms


In [ ]:
print('SCIP solution')
extract_solution(m_scip)

SCIP solution


{'assignment':          T1   T2   T3
 ground  1.0  0.0  4.0
 air     0.0  2.0  0.0
 sea     1.0  0.0  0.0,
 'survival_value': 4.95}

Lets look at the solution using `mindtpy`.
Because we dont want to do understand `mindtpy` functionalities more deeply, we simply hardcoded to use `cbc` as mip solver and `ipopt` as nlp solver.

in an upcoming blog we want to give a more details on the usages of `mindtpy`.

In [ ]:
data['solver'] = 'cbc/ipopt'

In [ ]:
%%time
m_cbc_ipopt = wta(data)

ERROR: Unable to clone Pyomo component attribute. Component 'I' contains an
    uncopyable field '_init_values' (<class
    'pyomo.core.base.set.TuplizeValuesInitializer'>).  Setting field to `None`
    on new object
ERROR: Unable to clone Pyomo component attribute. Component 'J' contains an
    uncopyable field '_init_values' (<class
    'pyomo.core.base.set.TuplizeValuesInitializer'>).  Setting field to `None`
    on new object
CPU times: user 128 ms, sys: 103 ms, total: 231 ms
Wall time: 510 ms


In [ ]:
print('cbc/ipopt solution')
extract_solution(m_cbc_ipopt)

cbc/ipopt solution


{'assignment':          T1   T2   T3
 ground  1.0  0.0  4.0
 air     0.0  2.0  0.0
 sea     1.0  0.0  0.0,
 'survival_value': 4.95}

## Summary

We saw how an non linear assigment problem can be implemented and solved using pyomo.
We solved the MINLP in two ways: 
- using a solver which can be applied to MINLP directly
- using the MINLP decomposition toolbox `mindtpy`, in our case using Coin-or branch and cut solver `cbc` "for the MIP" and the Interior Point OPTimizer `ipopt` "for the NLP".